In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import re

trainDF = pd.read_csv("train.csv")
trafoDF = pd.read_csv("trafo.csv")
submissionDF = pd.read_csv("submission.csv")
bagilNemDF = pd.read_csv("Hava Durumu/Bağıl Nem.csv")
bulutsuzlukOraniDF = pd.read_csv("Hava Durumu/Bulutluluk Oranı.csv")
radyasyonDF = pd.read_csv("Hava Durumu/Radyasyon.csv")
ruzgarHiziDF = pd.read_csv("Hava Durumu/Rüzgar Hızı.csv")
ruzgarYonuDF = pd.read_csv("Hava Durumu/Rüzgar Yönü.csv")
sicaklikDF = pd.read_csv("Hava Durumu/Sıcaklık.csv")
yagisDF = pd.read_csv("Hava Durumu/Yağış.csv")

radyasyonDF.drop("Unnamed: 17",axis=1,inplace=True) # bilinmeyen kolon ve içeriği boş olduğu için drop ediyoruz
trainDF.drop("BİLDİRİME_GÖRE",axis=1,inplace=True) # Bu kolon için tüm degerler aynı,bir katkısı olmayacak bu sebeple drop ediyoruz


def tr_capitalize(param_word):
    word_list = param_word.split(sep=" ")
    new_word = ""
    for word in word_list:
        first_letter = word[0]
        last_part = word[1:]

        first_letter = re.sub(r"i", "İ", first_letter)
        first_letter = re.sub(r"ı", "I", first_letter)
        first_letter = re.sub(r"ç", "Ç", first_letter)
        first_letter = re.sub(r"ş", "Ş", first_letter)
        first_letter = re.sub(r"ü", "Ü", first_letter)
        first_letter = re.sub(r"ğ", "Ğ", first_letter)

        last_part = re.sub(r"İ", "i", last_part)
        last_part = re.sub(r"I", "ı", last_part)
        last_part = re.sub(r"Ç", "ç", last_part)
        last_part = re.sub(r"Ş", "ş", last_part)
        last_part = re.sub(r"Ü", "ü", last_part)
        last_part = re.sub(r"Ğ", "ğ", last_part)

        rebuilt_word = first_letter + last_part
        rebuilt_word = rebuilt_word.capitalize()
        new_word = new_word + " " + rebuilt_word

    new_word = new_word.strip()
    return new_word

    
trainDF["BAŞLAMA_TARİHİ"] = pd.to_datetime(trainDF["BAŞLAMA_TARİHİ_VE_ZAMANI"], dayfirst=True).dt.date
trainDF["SONA_ERME_TARİHİ"] = pd.to_datetime(trainDF["SONA_ERME_TARİHİ_VE_ZAMANI"], dayfirst=True).dt.date

bagilNemDF["Tarih"]=pd.to_datetime(bagilNemDF["Tarih"],errors='coerce').dt.date
radyasyonDF["Tarih"]=pd.to_datetime(radyasyonDF["Tarih"],errors='coerce').dt.date
bulutsuzlukOraniDF["Tarih"]=pd.to_datetime(bulutsuzlukOraniDF["Tarih"],errors='coerce').dt.date
ruzgarHiziDF["Tarih"]=pd.to_datetime(ruzgarHiziDF["Tarih"],errors='coerce').dt.date
ruzgarYonuDF["Tarih"]=pd.to_datetime(ruzgarYonuDF["Tarih"],errors='coerce').dt.date
sicaklikDF["Tarih"]=pd.to_datetime(sicaklikDF["Tarih"],errors='coerce').dt.date
yagisDF["Tarih"]=pd.to_datetime(yagisDF["Tarih"],errors='coerce').dt.date


minDate = trainDF["BAŞLAMA_TARİHİ"].min()

# hava durumu ile ilgili dataframelerin tarihini traindf nin en düşük tarihine göre filtreliyoruz
bagilNemDF=bagilNemDF[bagilNemDF["Tarih"] >= minDate]
bulutsuzlukOraniDF=bulutsuzlukOraniDF[bulutsuzlukOraniDF["Tarih"] >= minDate]
radyasyonDF=radyasyonDF[radyasyonDF["Tarih"] >= minDate]
ruzgarHiziDF=ruzgarHiziDF[ruzgarHiziDF["Tarih"] >= minDate]
ruzgarYonuDF=ruzgarYonuDF[ruzgarYonuDF["Tarih"] >= minDate]
sicaklikDF=sicaklikDF[sicaklikDF["Tarih"] >= minDate]
yagisDF=yagisDF[yagisDF["Tarih"] >= minDate]


trafo_date=submissionDF["trafo_id_date"].str.split("__",expand=True)
submissionDF["trafo_id"]  = trafo_date[0]
submissionDF["Tarih"]  = trafo_date[1]
submissionDF.drop("trafo_id_date",axis=1,inplace=True)


In [8]:
havaDurumuLokasyonlar = list(set(itertools.chain(bagilNemDF.columns,bulutsuzlukOraniDF.columns,radyasyonDF.columns,ruzgarHiziDF.columns,ruzgarYonuDF.columns,sicaklikDF.columns,yagisDF.columns)))

trainDF["İL"]=trainDF["İL"].apply(lambda x: tr_capitalize(x))
trainDF["İLÇE"]=trainDF["İLÇE"].apply(lambda x: tr_capitalize(x))
trainDF["LOKASYON"] = trainDF[["İL","İLÇE"]].apply(lambda x: x["İLÇE"] if x["İLÇE"] in havaDurumuLokasyonlar else x["İL"], axis=1)
trainDF.drop(["İL","İLÇE"],axis=1,inplace=True)

In [ ]:
# her saatır için lokasyon(il ve ilçeye göre) ve tarihe göre hava durumu ortalama degerlerini yeni bir kolonda gösteriyoruz
for dfDegisken in [bagilNemDF,bulutsuzlukOraniDF,radyasyonDF,ruzgarHiziDF,ruzgarYonuDF,sicaklikDF,yagisDF]:
    ortalama = dfDegisken.groupby("Tarih").mean().to_dict()
    tarihler=dfDegisken.groupby("Tarih").mean().index
    if(str(dfDegisken)==str(bagilNemDF)):
        kolon_adi="BAĞIL_NEM"
    elif(str(dfDegisken)==str(bulutsuzlukOraniDF)):
        kolon_adi="BULUTSUZLUK_ORANI"
    elif(str(dfDegisken)==str(radyasyonDF)):
        kolon_adi="RADYASYON"
    elif(str(dfDegisken)==str(ruzgarHiziDF)):
        kolon_adi="RUZGAR_HIZI"
    elif(str(dfDegisken)==str(ruzgarYonuDF)):
        kolon_adi="RUZGAR_YONU"
    elif(str(dfDegisken)==str(sicaklikDF)):
        kolon_adi="SICAKLIK"
    elif(str(dfDegisken)==str(yagisDF)):
        kolon_adi="YAGIS"
    
    trainDF[kolon_adi] = trainDF.apply(lambda x: ortalama[x["LOKASYON"]][x["BAŞLAMA_TARİHİ"]] if x["BAŞLAMA_TARİHİ"] in tarihler else 0.0,axis=1)

In [9]:
trainDF=pd.merge(trainDF,trafoDF,on='ŞEBEKE_UNSURU_KODU',how='left')
trainDF['target']=1
#start -> trainDF["BAŞLAMA_TARİHİ"]).min() and end -> trainDF["BAŞLAMA_TARİHİ"]).max()
finalDF=pd.DataFrame(pd.date_range(start='5/1/2021', end='6/12/2022', freq='D'))
finalDF.rename(columns={0:"Tarih"},inplace=True)

uniqueTrafo=pd.DataFrame()
uniqueTrafo["trafo_id"]=trainDF["trafo_id"].unique()
uniqueTrafo=uniqueTrafo.merge(trainDF[["trafo_id","LOKASYON"]],on="trafo_id",how="inner").drop_duplicates()

finalDF=finalDF.merge(uniqueTrafo[["trafo_id","LOKASYON"]], how='cross')

trainDF.rename(columns={"BAŞLAMA_TARİHİ":"Tarih"},inplace=True)
trainDF["Tarih"]=trainDF["Tarih"].astype('datetime64[ns]')



,Tarih,trafo_id,LOKASYON
0,2021-05-01,0,Urla
1,2021-05-01,1,İzmir
2,2021-05-01,2,İzmir
3,2021-05-01,3,Bergama
4,2021-05-01,4,İzmir
...,...,...,...
26007139,2022-06-12,63762,İzmir
26007140,2022-06-12,63763,İzmir
26007141,2022-06-12,63764,İzmir
26007142,2022-06-12,63765,Akhisar


In [10]:
finalDF = pd.merge(finalDF,trainDF, on=["Tarih",'trafo_id'],how="left")

finalDF[finalDF["KESİNTİ_NO"].isna()]
finalDF






c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexes\base.py:5010: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return key in self._engine


KeyboardInterrupt: 

,Tarih,İzmir,Aliağa,Çeşme,Bergama,Manisa,Akhisar,Salihli,Çiğli,Menderes,Alaşehir,Demirci,Güzelyalı,Kula,Ödemiş,Soma,Urla
2136,2021-05-01,0.005,0.009,0.010,0.005,0.005,0.006,0.004,0.009,0.004,0.005,0.004,0.005,0.004,0.005,0.004,0.009
2137,2021-05-01,0.005,0.009,0.010,0.004,0.005,0.005,0.004,0.009,0.004,0.005,0.004,0.005,0.004,0.004,0.004,0.009
2138,2021-05-01,0.005,0.009,0.010,0.004,0.004,0.005,0.003,0.009,0.004,0.004,0.004,0.005,0.004,0.004,0.004,0.009
2139,2021-05-01,0.005,0.009,0.011,0.004,0.004,0.006,0.003,0.009,0.004,0.004,0.005,0.005,0.004,0.004,0.005,0.009
2140,2021-05-01,0.005,0.009,0.010,0.004,0.004,0.005,0.003,0.009,0.004,0.004,0.005,0.005,0.004,0.004,0.005,0.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11779,2022-06-06,0.011,0.011,0.015,0.012,0.014,0.010,0.015,0.011,0.010,0.010,0.008,0.011,0.009,0.009,0.009,0.012
11780,2022-06-06,0.011,0.011,0.015,0.011,0.014,0.010,0.013,0.011,0.010,0.010,0.008,0.011,0.009,0.009,0.009,0.012
11781,2022-06-06,0.011,0.012,0.014,0.010,0.013,0.009,0.011,0.011,0.010,0.009,0.009,0.011,0.008,0.009,0.009,0.012
11782,2022-06-06,0.010,0.011,0.014,0.010,0.012,0.009,0.011,0.010,0.010,0.009,0.009,0.010,0.009,0.009,0.010,0.012
